In [ ]:
import time
import pandas as pd
from dataclasses import dataclass

from xml.parsers.expat import ExpatError

from youtube_transcript_api import YouTubeTranscriptApi as YTTranscriptApi
from youtube_transcript_api._errors import NoTranscriptFound

data = pd.read_parquet("../data/videos-info.parquet")

In [ ]:
@dataclass
class Transcript:
    id: str
    text: str

In [ ]:
transcripts = []

for row in data[["id"]].itertuples(index=False):
    time.sleep(0.5)  # Avoid hitting the API too fast
    try:
        transcript = YTTranscriptApi.get_transcript(row.id, languages=["en", "en-US"])
        full_transcript = " ".join([piece["text"] for piece in transcript])
        transcripts.append(Transcript(row.id, full_transcript))
    except NoTranscriptFound as e:
        print(f"No transcript found for {row.id}: {e}")
    except TypeError as e:
        print(f"TypeError for {row.id}: {e}")
    except ExpatError as e:
        print(f"ExpatError for {row.id}: {e}")
    except Exception as e:
        print(f"Unexpected error for {row.id}: {e}")
        continue

In [ ]:
transcript = YTTranscriptApi.get_transcript("PkZNo7MFNFg")

In [ ]:
pd.DataFrame(transcripts).to_parquet(
    "../data/transcripts.parquet",
    index=False,
    engine="pyarrow",
    compression="snappy",
)